<a href="https://colab.research.google.com/github/rishabhs-s/churn-modelling-with-DL/blob/master/Churn_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
from google.colab import drive

In [107]:

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
import pandas as pd
import numpy as np
import keras

In [109]:
data=pd.read_csv("/content/drive/My Drive/Deep Learning/Churn_Modelling.csv")

In [110]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [112]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [113]:
X=data.iloc[:,3:13]
y=data.iloc[:,13]

In [114]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [115]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [116]:
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X["Gender"],drop_first=True)



In [117]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [118]:
X.drop(['Geography','Gender'],axis=1,inplace=True)


In [119]:
X=pd.concat([X,gender,geography],axis=1)

In [120]:
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,0
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,0
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,0


In [121]:
geography.head(600)

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1
...,...,...
595,1,0
596,0,1
597,1,0
598,1,0


In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [123]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_train=ss.fit_transform(X_train)
X_test==ss.fit_transform(X_test)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male,Germany,Spain
2854,False,False,False,False,False,False,False,False,False,False,False
3403,False,False,False,False,False,False,False,False,False,False,False
6357,False,False,False,False,False,False,False,False,False,False,False
6165,False,False,False,False,False,False,False,False,False,False,False
7761,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
9874,False,False,False,False,False,False,False,False,False,False,False
2555,False,False,False,False,False,False,False,False,False,False,False
1935,False,False,False,False,False,False,False,False,False,False,False
9046,False,False,False,False,False,False,False,False,False,False,False


In [153]:
import keras
from keras.models import Sequential
from keras.layers import Dropout,Dense

In [154]:
model=Sequential()

In [155]:
#1st layer
model.add(Dense(units=6,input_dim=11,kernel_initializer='he_uniform',activation='relu'))


In [156]:
#2nd layer
model.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
#model.add(Dropout(0.2))

In [157]:
#o/p layer
model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [158]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [159]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_history=model.fit(X_train,y_train,batch_size=10,validation_split=0.33,epochs=100)


Epoch 1/100
503/503 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7741 - val_loss: 0.4959 - val_accuracy: 0.7924
Epoch 2/100
503/503 [==============================] - 1s 1ms/step - loss: 0.4639 - accuracy: 0.7994 - val_loss: 0.4695 - val_accuracy: 0.7928
Epoch 3/100
503/503 [==============================] - 1s 1ms/step - loss: 0.4387 - accuracy: 0.8055 - val_loss: 0.4527 - val_accuracy: 0.7964
Epoch 4/100
503/503 [==============================] - 1s 1ms/step - loss: 0.4231 - accuracy: 0.8131 - val_loss: 0.4412 - val_accuracy: 0.8053
Epoch 5/100
503/503 [==============================] - 1s 1ms/step - loss: 0.4096 - accuracy: 0.8217 - val_loss: 0.4286 - val_accuracy: 0.8114
Epoch 6/100
503/503 [==============================] - 1s 1ms/step - loss: 0.3961 - accuracy: 0.8306 - val_loss: 0.4152 - val_accuracy: 0.8215
Epoch 7/100
503/503 [==============================] - 1s 1ms/step - loss: 0.3846 - accuracy: 0.8384 - val_loss: 0.4018 - val_accuracy: 0.8348

In [147]:
y_pred = model.predict(X_test)

In [148]:
y_pred = (y_pred > 0.5)

In [149]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)


In [150]:
cm

array([[1990,    0],
       [ 510,    0]])

In [151]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

In [152]:
score

0.796